# Get a list of Trove newspapers that doesn't include government gazettes

The Trove API includes an option to retrieve details of digitised newspaper titles. Version 2 of the API added a separate option to get details of government gazettes. However the original `newspaper/titles` requests actually returns *both* the newspaper and gazette titles, so there's no way of getting just the newspaper titles. This notebook explains the problem and provides a simple workaround.

In [2]:
import requests
import pandas as pd

Add your Trove API key below.

In [3]:
api_key = '6pi5hht0d2umqcro'
print('Your API key is: {}'.format(api_key))

Your API key is: 6pi5hht0d2umqcro


## The problem

Getting a list of digitised newspapers or gazettes in Trove is easy, you just fire off a request to one of these endpoints:

* `https://api.trove.nla.gov.au/v2/newspaper/titles/`
* `https://api.trove.nla.gov.au/v2/gazette/titles/`

Let's create a function to get either the `newspaper` or `gazette` results.

In [6]:
def get_titles_df(title_type):
    # Set default params
    params = {
        'key': api_key,
        'encoding': 'json',
    }
    
    # Make the request to the titles endpoint and get the JSON data
    data = requests.get('https://api.trove.nla.gov.au/v2/{}/titles'.format(title_type), params=params).json()
    titles = []
    
    # Loop through the title records, saving the name and id
    for title in data['response']['records']['newspaper']:
        titles.append({'title': title['title'], 'id': int(title['id'])})
        
    # Convert to a dataframe
    df = pd.DataFrame(titles)
    return df

Let's use the function to get all the newspaper titles.

In [10]:
newspapers_df = get_titles_df('newspaper')
newspapers_df.head()

,id,title
0,166,Canberra Community News (ACT : 1925 - 1927)
1,165,Canberra Illustrated: A Quarterly Magazine (AC...
2,69,"Federal Capital Pioneer (Canberra, ACT : 1924 ..."
3,871,Good Neighbour (ACT : 1950 - 1969)
4,665,Student Notes/Canberra University College Stud...


How many are there?

In [8]:
newspapers_df.shape

(1454, 2)

Everything looks ok, but if we search inside the results for titles that include the word 'Gazette' we find that the government gazettes are all included.

In [11]:
newspapers_df.loc[newspapers_df['title'].str.contains('Gazette')][:20]

,id,title
12,1372,Papua New Guinea Government Gazette (1971 - 1975)
17,1371,Territory of Papua and New Guinea Government G...
18,1369,Territory of Papua Government Gazette (Papua N...
19,1370,Territory of Papua-New Guinea Government Gazet...
21,1288,Australian Government Gazette (National : 1973...
22,1355,Australian Government Gazette. Chemical (Natio...
23,1289,Australian Government Gazette. General (Nation...
24,1294,Australian Government Gazette. Periodic (Natio...
25,1308,Australian Government Gazette. Public Service ...
26,1286,Australian Government Gazette. Special (Nation...


## The solution

We can't just filter the results on the word 'Gazette' as a number of newspapers also include the word in their titles. Instead, we'll get a list of the gazettes using the `gazette/titles` endpoint and subtract these titles from the list of newspapers.

Let's get the gazettes.

In [12]:
gazettes_df = get_titles_df('gazette')
gazettes_df.head()

,id,title
0,1372,Papua New Guinea Government Gazette (1971 - 1975)
1,1371,Territory of Papua and New Guinea Government G...
2,1369,Territory of Papua Government Gazette (Papua N...
3,1370,Territory of Papua-New Guinea Government Gazet...
4,1288,Australian Government Gazette (National : 1973...


In [13]:
gazettes_df.shape

(37, 2)

Now we'll create a new dataframe that only includes titles from `df_newspapers` if they **are not in** `df_gazettes`.

In [15]:
newspapers_not_gazettes_df = newspapers_df[~newspapers_df['id'].isin(gazettes_df['id'])]

In [16]:
newspapers_not_gazettes_df.shape

(1417, 2)

If it worked properly the number of titles in the new dataframe should equal the number in the newspapers dataframe minus the number in the gazettes dataframe.

In [17]:
newspapers_not_gazettes_df.shape[0] == newspapers_df.shape[0] - gazettes_df.shape[0]

True

Yay!